# In this notebook we analyze quality of a tracker conditioned on depth

In [ ]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np 
from PIL import Image

### Compare a median tracker with inferred depth vs ground truth

In [ ]:
gt_depth_tracker = pd.read_csv('../data/interim/median_gt_depth.csv')
inferred_depth_tracker = pd.read_csv('../data/interim/median_adabins_motsynth.csv')

In [ ]:
n = gt_depth_tracker['seq'].shape[0]
plt.plot(np.arange(n), gt_depth_tracker['HOTA(0)'] * 100, label='GT depth')
plt.plot(np.arange(n), inferred_depth_tracker['HOTA(0)'] * 100, label='Inferred depth')
plt.legend()
plt.show()

In [ ]:
diff = gt_depth_tracker['HOTA(0)'] - inferred_depth_tracker['HOTA(0)']
plt.plot(np.arange(n), diff * 100)
plt.show()

In [ ]:
plt.hist(np.array(diff) * 100, bins=100);

In [ ]:
gt_depth_tracker.nsmallest(10, 'HOTA(0)')

In [ ]:
inferred_depth_tracker.nsmallest(10, 'HOTA(0)')

In [ ]:
print("Worst gt HOTA joined on inferred HOTA")
gt_depth_tracker.join(
    inferred_depth_tracker.set_index('seq'), 
    on='seq', 
    how='left', 
    lsuffix='_gt', 
    rsuffix='_inferred')[['seq', 'HOTA(0)_gt', 'HOTA(0)_inferred']].nsmallest(10, 'HOTA(0)_gt')

In [ ]:
print("Worst inferred HOTA joined on gt HOTA")
inferred_depth_tracker.join(
    gt_depth_tracker.set_index('seq'), 
    on='seq', 
    how='left', 
    lsuffix='_inferred', 
    rsuffix='_gt')[['seq', 'HOTA(0)_inferred', 'HOTA(0)_gt']].nsmallest(20, 'HOTA(0)_inferred')

In [ ]:
# We want to find sequences with the largest degradation and fix them 
largest_diff = pd.concat([gt_depth_tracker['seq'], diff], axis=1).sort_values('HOTA(0)', ascending=False)

In [ ]:
largest_diff[:20]

In [ ]:
for s in largest_diff[:20]['seq']:
    print("\"{}\", ".format(s), end='')